In [ ]:
import ginza
stopwords = list(ginza.STOP_WORDS)

In [ ]:
print(stopwords)

In [ ]:
import urllib.request, urllib.error
slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(slothlib_path)
slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']

In [ ]:
print(slothlib_stopwords)
print('こんにちは' not in slothlib_stopwords)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import spacy
nlp = spacy.load('ja_ginza')
from parse_5w1h import parse_5w1h
#テストファイル読み込み&下処理
path = "/Users/shota/Documents/itac/test_summary1.txt"
with open(path) as f:
    text = f.read()
text = text.replace('\n','')
print(text)
doc = nlp(text)

In [ ]:
import networkx as nx

def increment_edge (graph, node0, node1):
    print("link {} {}".format(node0, node1))
    
    if graph.has_edge(node0, node1):
        graph[node0][node1]["weight"] += 1.0
    else:
        graph.add_edge(node0, node1, weight=1.0)

In [ ]:
POS_KEPT = ["ADJ", "NOUN", "PROPN", "VERB"]

def link_sentence (doc, sent, lemma_graph, seen_lemma):
    visited_tokens = []
    visited_nodes = []

    for i in range(sent.start, sent.end):
        token = doc[i]

        if token.pos_ in POS_KEPT and token.lemma_ not in slothlib_stopwords:
            #token.lemma_は原型. token.pos_は品詞
            key = (token.lemma_, token.pos_)

            if key not in seen_lemma:
                seen_lemma[key] = set([token.i])
            else:
                seen_lemma[key].add(token.i)

            node_id = list(seen_lemma.keys()).index(key)

            if not node_id in lemma_graph:
                lemma_graph.add_node(node_id)

            print("visit {} {}".format(visited_tokens, visited_nodes))
            print("range {}".format(list(range(len(visited_tokens) - 1, -1, -1))))
            
            for prev_token in range(len(visited_tokens) - 1, -1, -1):
                print("prev_tok {} {}".format(prev_token, (token.i - visited_tokens[prev_token])))
                
                if (token.i - visited_tokens[prev_token]) <= 3:
                    increment_edge(lemma_graph, node_id, visited_nodes[prev_token])
                else:
                    break

            print(" -- {} {} {} {} {} {}".format(token.i, token.text, token.lemma_, token.pos_, visited_tokens, visited_nodes))

            visited_tokens.append(token.i)
            visited_nodes.append(node_id)

In [ ]:
lemma_graph = nx.Graph()
seen_lemma = {}

for sent in doc.sents:
    link_sentence(doc, sent, lemma_graph, seen_lemma)
    #break # only test one sentence
labels = {}
keys = list(seen_lemma.keys())

for i in range(len(seen_lemma)):
    labels[i] = keys[i][0].lower()
ranks = nx.pagerank(lemma_graph)
imp_list = []
for node_id, rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True):
    imp_list.append(labels[node_id])

In [ ]:
import math
def collect_5w1hphrases (chunk,chunk_start,chunk_end, phrases, counts):
    chunk_len = chunk_end - chunk_start + 1
    sq_sum_rank = 0.0
    non_lemma = 0
    compound_key = set([])

    for i in range(chunk_start, chunk_end):
        token = doc[i]
        key = (token.lemma_, token.pos_)
        
        if key in seen_lemma:
            node_id = list(seen_lemma.keys()).index(key)
            rank = ranks[node_id]
            sq_sum_rank += rank
            compound_key.add(key)
        
            #print(" {} {} {} {}".format(token.lemma_, token.pos_, node_id, rank))
        else:
            non_lemma += 1
    
    # although the noun chunking is greedy, we discount the ranks using a
    # point estimate based on the number of non-lemma tokens within a phrase
    non_lemma_discount = chunk_len / (chunk_len + (2.0 * non_lemma) + 1.0)

    # use root mean square (RMS) to normalize the contributions of all the tokens
    phrase_rank = math.sqrt(sq_sum_rank / (chunk_len + non_lemma))
    phrase_rank *= non_lemma_discount

    # remove spurious punctuation
    phrase = chunk.replace("'", "")

    # create a unique key for the the phrase based on its lemma components
    compound_key = tuple(sorted(list(compound_key)))
    
    if not compound_key in phrases:
        phrases[compound_key] = set([ (phrase, phrase_rank) ])
        counts[compound_key] = 1
    else:
        phrases[compound_key].add( (phrase, phrase_rank) )
        counts[compound_key] += 1

    #print("{} {} {} {} {} {}".format(phrase_rank, chunk, chunk_start, chunk_end, chunk_len, counts[compound_key]))

In [ ]:
from parse_5w1h import parse_5w1h
parse = parse_5w1h(0)
parse.extract(text)

In [ ]:
parse.display_5w1h()

In [ ]:
phrases = {}
counts = {}

for i,chunk in enumerate(parse._5w1hs):

     collect_5w1hphrases(chunk, parse._5w1h_s[i],parse._5w1h_e[i],phrases, counts)
  

In [ ]:
import operator

min_phrases = {}

for compound_key, rank_tuples in phrases.items():
    l = list(rank_tuples)
    l.sort(key=operator.itemgetter(1), reverse=True)
    
    phrase, rank = l[0]
    count = counts[compound_key]
    
    min_phrases[phrase] = (rank, count)

In [ ]:
min_phrases[phrase][0]

In [ ]:
i = 0
for phrase, (rank, count) in min_phrases.items():

    if rank > 0.05:
        #summary.append(phrase)
        if parse.doc[parse._5w1h_e[i]]._._5w1h == 'How':
            l = i
            while l > 0:
                print(parse._5w1hs[l-1])
                summary.append(parse._5w1hs[l-1])
                if parse.doc[parse._5w1h_e[l-1]]._._5w1h == 'Who'or parse.doc[parse._5w1h_e[l-1]]._._5w1h == 'What':
                    break
                l=l-1
        else:
            l = i
            while l < len(parse._5w1hs)-1:
                summary.append(parse._5w1hs[l+1])
                if parse.doc[parse._5w1h_e[l+1]]._._5w1h == 'How':
                    break
                l=l+1
                
        print(''.join(summary))
    i = i +1

In [ ]:
for i,chunk in enumerate(parse._5w1hs):
    summary = []
    rank = collect_5w1hphrases(chunk, parse._5w1h_s[i],parse._5w1h_e[i],phrases, counts)
    #print(parse.doc[parse._5w1h_e[i]-1]._._5w1h)
    if rank > 0.05:
        summary.append(chunk)
        print(chunk)
        print(parse.doc[parse._5w1h_e[i]]._._5w1h)
        if parse.doc[parse._5w1h_e[i]]._._5w1h == 'How':
            l = i
            while l > 0:
                
                summary.append(parse._5w1hs[l-1])
                if parse.doc[parse._5w1h_e[l-1]]._._5w1h == 'Who'or parse.doc[parse._5w1h_e[l-1]]._._5w1h == 'What':
                    break
                l=l-1
        else:
            l = i
            while l < len(parse._5w1hs)-1:
                summary.append(parse._5w1hs[l+1])
                if parse.doc[parse._5w1h_e[l+1]]._._5w1h == 'How':
                    break
                l=l+1
                
        #print(''.join(summary))
        
        

In [ ]:
for phrase, (rank, count) in sorted(min_phrases.items(), key=lambda x: x[1][0], reverse=True)[0:10]:
    print(phrase)

In [ ]:
for phrase, (rank, count) in min_phrases.items():
    print(phrase)
    print(rank)

In [ ]:
len(min_phrases.items())

In [ ]:
min_phrases.items()[1]

In [ ]:
import math
def collect_phrases (chunk, phrases, counts):
    chunk_len = chunk.end - chunk.start + 1
    sq_sum_rank = 0.0
    non_lemma = 0
    compound_key = set([])

    for i in range(chunk.start, chunk.end):
        token = doc[i]
        key = (token.lemma_, token.pos_)
        
        if key in seen_lemma:
            node_id = list(seen_lemma.keys()).index(key)
            rank = ranks[node_id]
            sq_sum_rank += rank
            compound_key.add(key)
        
            print(" {} {} {} {}".format(token.lemma_, token.pos_, node_id, rank))
        else:
            non_lemma += 1
    
    # although the noun chunking is greedy, we discount the ranks using a
    # point estimate based on the number of non-lemma tokens within a phrase
    non_lemma_discount = chunk_len / (chunk_len + (2.0 * non_lemma) + 1.0)

    # use root mean square (RMS) to normalize the contributions of all the tokens
    phrase_rank = math.sqrt(sq_sum_rank / (chunk_len + non_lemma))
    phrase_rank *= non_lemma_discount

    # remove spurious punctuation
    phrase = chunk.text.lower().replace("'", "")

    # create a unique key for the the phrase based on its lemma components
    compound_key = tuple(sorted(list(compound_key)))
    
    if not compound_key in phrases:
        phrases[compound_key] = set([ (phrase, phrase_rank) ])
        counts[compound_key] = 1
    else:
        phrases[compound_key].add( (phrase, phrase_rank) )
        counts[compound_key] += 1

    print("{} {} {} {} {} {}".format(phrase_rank, chunk.text, chunk.start, chunk.end, chunk_len, counts[compound_key]))

In [ ]:
phrases = {}
counts = {}
for sent in doc.sents:
    collect_phrases(sent, phrases, counts)

In [ ]:
import operator

min_phrases = {}

for compound_key, rank_tuples in phrases.items():
    l = list(rank_tuples)
    l.sort(key=operator.itemgetter(1), reverse=True)
    
    phrase, rank = l[0]
    count = counts[compound_key]
    
    min_phrases[phrase] = (rank, count)

In [ ]:
for phrase, (rank, count) in sorted(min_phrases.items(), key=lambda x: x[1][0], reverse=True)[0:3]:
    print(phrase)